# CSVファイルの読み込みを行う

移植成功

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import json

class CSVProcessor:
    def __init__(self, input_file, output_file):
        self.input_file = input_file
        self.output_file = output_file

    def generate_id(self,datas):
        datas['ID']=datas.apply(lambda row:self.generate_single_id(row['Date'],row['Amount'],row.get('ID'),row['remark']),axis=1)

    def generate_single_id(self, date:str ,amount:int,remark, existing_id:int=None)->int:
        """
        Generate a unique ID based on the date and amount.

        Args:
            date (str): The date of the transaction in "YYYY-MM-DD" format.
            amount (float): The amount of the transaction.
            existing_id (int, optional): An existing ID. If specified, returns this ID.

        Returns:
            int: The generated ID.
        """
        if np.isnan(existing_id)or existing_id is None:
            date_str = str(date)
            date_part = date_str.replace("-","")
            sign = "1" if float(amount) >= 0 else "0"
            remark_suffix = remark[-2:] if len(remark) >= 2 else remark
            # IDを生成し、Remarkの後ろ2文字を連結
            generated_id = f"{date_part}{sign}{remark_suffix}"
            return int(generated_id)
        else:
            return int(existing_id)



    def fill_remarks(self):
        """
        備考欄を埋める
        """
        pass

    def sort_csv(self,datas):
        """
        CSVファイルをIDでソートする
        """
        datas_sorted = datas.sort_values(by='ID')
        return datas_sorted

    def add_yearmonth_column(self,datas):
        """
        日付列から年月を抽出して新しい列を追加する
        """
        datas['Date']=pd.to_datetime(datas['Date'])
        datas['Year']=datas['Date'].dt.year
        datas['Month']=datas['Date'].dt.month
        return datas

    def find_by_id(self,file_path, search_id):
        search_id=str(search_id)
        try:
            # JSONファイルを読み込む
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
        except FileNotFoundError:
            print(f"Error: File '{file_path}' not found.")
            return None
        except json.JSONDecodeError:
            print(f"Error: File '{file_path}' is not a valid JSON.")
            return None

        # 指定されたidを持つ要素を検索する
        for key, value in data.items():
            # valueが辞書であり、'id'キーが存在するかどうかを確認
            if isinstance(value, dict) and 'id' in value:
                if value['id'] == search_id:
                    return key

        print(f"Error: ID '{search_id}' not found in JSON.")
        return None

    def process_csv(self):
        """
        CSVファイルの処理を行う
        """
        datas = pd.read_csv(self.input_file)

        # IDを生成する
        datas["ID"] = datas.apply(lambda data:self.generate_single_id(data["Date"],data["Amount"],data['Remarks'],data.get("ID")),axis=1)
        file_path = 'codes.json'  # JSONファイルのパス
        datas['Subject'] = datas.apply(lambda data:self.find_by_id(file_path, data['SubjectCode']),axis=1)

        # CSVファイルをIDでソートする
        datas = self.sort_csv(datas)

        # 日付列から月を抽出して新しい列を追加する
        datas = self.add_yearmonth_column(datas)
        # JSONから科目を引用する

        datas = datas.drop_duplicates(subset='ID',keep='first')

        print(datas)

        # 処理されたデータを新しいCSVファイルに保存する
        datas.to_csv(self.output_file, index=False)

# オブジェクトの作成と処理の実行

csv_file_name = 'datas/financial_records.csv'
datas = pd.read_csv(csv_file_name)
processor = CSVProcessor(csv_file_name, 'datas/financial_records_modified.csv')
processor.process_csv()


Error: ID '260' not found in JSON.
Error: ID '270' not found in JSON.
Error: ID '260' not found in JSON.
Error: ID '270' not found in JSON.
Error: ID '260' not found in JSON.
Error: ID '270' not found in JSON.
          Date           ID  SubjectCode  Amount        Remarks  \
197 2024-04-01   2024041099          130   -1770  Carry Over 99   
196 2024-04-01   2024041199          111   20000  Carry Over 99   
198 2024-04-03   2024043001          101   -2000      PayPay 01   
199 2024-04-03   2024043101          500    2000      PayPay 01   
200 2024-04-06   2024046001          100    -775        Thigh01   
..         ...          ...          ...     ...            ...   
407 2024-06-05  20240605102          530     900     Suishun 02   
412 2024-06-06  20240606002          101   -1299      Amazon 02   
411 2024-06-06  20240606102          513    1299      Amazon 02   
414 2024-06-08  20240608001          100   -1000          Gym01   
413 2024-06-08  20240608101          580    1000     

### For test
some method

In [9]:
csv_file_path = 'datas/test.csv'
processor = CSVProcessor(csv_file_path, 'datas/test_modified.csv')
processor.process_csv()

         Date           ID  SubjectCode  Amount       Remarks  \
0  2024-03-03  20240303001          200   -2000  Starbucks 01   
2  2024-03-03  20240303002          101   -2145    Tempura 02   
4  2024-03-03  20240303003          101   -2896       Plug 03   
1  2024-03-03  20240303101          500    2000  Starbucks 01   
3  2024-03-03  20240303102          531    2145    Tempura 02   
..        ...          ...          ...     ...           ...   
50 2024-04-16  20240416002          101   -3000     PayPay 02   
52 2024-04-16  20240416003          101   -3100      salon 03   
49 2024-04-16  20240416101          531    1410   yakiniku 01   
51 2024-04-16  20240416102          531    3000     PayPay 02   
53 2024-04-16  20240416103          542    3100      salon 03   

             Subject  Year  Month  
0   Credit Card Debt  2024      3  
2                UFJ  2024      3  
4                UFJ  2024      3  
1          Groceries  2024      3  
3         Dining Out  2024      3  
.. 

各項目で合計が一致しない時にエラーを吐くメソッド //
1. IDの最後のケタだけを除いてAmountを合計する
2. ０でないときにそのIDを出力する 
3. 先にチェックしてからOKならそのまま変換していく
4. ダメならExitする

In [ ]:
def check_sum():

    return allow

繰り越し作業


月、年を表す列を追加する

In [52]:
import json

def find_by_id(file_path, search_id):
    try:
        # JSONファイルを読み込む
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: File '{file_path}' is not a valid JSON.")
        return None

    # 指定されたidを持つ要素を検索する
    for key, value in data.items():
        # valueが辞書であり、'id'キーが存在するかどうかを確認
        if isinstance(value, dict) and 'id' in value:
            if value['id'] == search_id:
                return key

    print(f"Error: ID '{search_id}' not found in JSON.")
    return None

# 使用例
file_path = 'codes.json'  # JSONファイルのパス
search_id = '100'  # 検索するID

result = find_by_id(file_path, search_id)

if result:
    print(f"Found entry: {result}")
else:
    print("No entry found with the specified ID.")

Found entry: Cash


In [9]:
import json
import pandas as pd

# codes.jsonファイルから科目表を読み込む
with open('codes.json', 'r') as f:
    codes = json.load(f)

# 科目コードとカテゴリを参照して、科目名とカテゴリ名を追加する関数
def get_subject_category(code):
    category = ''
    subject = ''
    for cat, subjects in codes.items():
        if code in subjects:
            category = cat
            subject = subjects[code]['name']
            break
    return category, subject

# CSVファイルのデータに科目名とカテゴリ名を追加する
datas['Category'], datas['Subject'] = zip(*datas['Subject Code'].map(get_subject_category))

# 結果を新しいCSVファイルに書き込む
datas.to_csv('financial_records_with_categories.csv', index=False)

print("CSVファイルにデータが書き込まれました。")


KeyError: 'Subject Code'